<a href="https://colab.research.google.com/github/sosanzma/Manel/blob/master/EDO1_Manel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !git clone https://github.com/facebookresearch/SymbolicMathematics
import os
os.chdir('SymbolicMathematics/')
!ls


fatal: destination path 'SymbolicMathematics' already exists and is not an empty directory.
beam_integration.ipynb	CONTRIBUTING.md  main.py   README.md	  src
CODE_OF_CONDUCT.md	LICENSE		 ode1.pth  split_data.py


In [ ]:
import numpy as np
import sympy as sp
import torch

from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify
from sympy import sympify
from sympy import checkodesol

In [ ]:
!wget https://dl.fbaipublicfiles.com/SymbolicMathematics/models/ode1.pth
model_path = './ode1.pth'
assert os.path.isfile(model_path)

--2021-04-06 09:01:27--  https://dl.fbaipublicfiles.com/SymbolicMathematics/models/ode1.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1849505529 (1.7G) [application/octet-stream]
Saving to: ‘ode1.pth.1’

ode1.pth.1          100%[===================>]   1.72G  12.6MB/s    in 2m 21s  

2021-04-06 09:03:49 (12.5 MB/s) - ‘ode1.pth.1’ saved [1849505529/1849505529]



In [ ]:
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 2,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})


In [ ]:
env = build_env(params)      # Estamos definiendo el entorno con el que vamos a trabajar, le metemos los parametros que queramos
x = env.local_dict['x']
y = env.local_dict['y']     ## crec que es refereix a la "variable d'estudi"

In [ ]:
modules = build_modules(env, params)  
encoder = modules['encoder']   # creamos el encoder a partir del entrono y los parametros
decoder = modules['decoder']    # creamos el decoder        "        "

In [ ]:
# here you can modify the the  solution to the equation that the model has to predict
#F_infix = 'sin(x) + a8*cos(x)'

In [ ]:
# F (la supuesta solucion a la EDO que definamos como input)
#F = sp.S(F_infix, locals=env.local_dict)   
#F        

# no necessitem estes funcions

Si queremos resolver una EDO del tipo y' + y = g(x) , el input tiene que ser de la forma : y' + y - g(x) 

La notació que se utiliza para la viariable diferenciada en función de x (en el ejemplo es : 'y' )  : f(x) 

Y la derivada se escribe de la forma : f'(x) --> diff(f(x), x)


In [ ]:
# f (F', that the model will take as input, edo a calcular)
f1 = 'cos(x)*diff(f(x),x) + sin(x)*f(x)-1 '     # equació de prova, dona bons resultats

f2 = 'diff(f(x),x) - 2*f(x) - 2*exp(2*sin(x))*(cos(x)-1)'   # esta dona bons resultats en la comprovació
f3 = 'diff(f(x),x)*x^2 + x*f(x) + sqrt(f(x))'   # el primer resultat es correcte, pero a l'hora de la comprovació no dona
f4 = 'diff((f(x))^2,x) + 2*(f(x))^2/x - 2*f(x)/(cos(x)^2)' #esta es complica a l'hora de comprovar-ho. No sé si dona bons results (massa complicada?)
f5= 'x^2*(diff(f(x),x)+ f(x)^2) + x*f(x) -1'     # esta dona bé també
f6 = 'diff(f(x),x) + 2*f(x)/x - (2*f(x))/(cos(x)^2)'  # esta és igual que la 4 però sense l'arrel. Trau resultat correcte i molt ràpidamaent. (problema en l'arrel al comprovar)
f7='diff(f(x),x)*(x- 2*f(x))+ 2*x + f(x)'  # no dona bé el resultat per un signe
f8='x*diff(f(x),x) - x^2*f(x)- exp(x^2/2)' # dona bé
f9= 'diff(f(x),x) - a0*f(x)+f(x)^3'   # si la constant que fique en f(x) es 1 apareix un error
f10= 'diff(f(x),x)*f(x)+ 1'  # esta ecuació és tan senzilla que deuria de ser capaç de resordre-la --> problemes en funcions en el denominador
f11='diff(f(x),x) - f(x)/x - tan(f(x)/x)' # dona un error de compilació
f12='diff(f(x),x) + 1/(2*f(x)*x)*(f(x)^2 + 2/x)' # no dona ( altra vegada te la funció en el denominador)
f13='diff(f(x),x)*x + f(x) - 2*x^4*f(x)^4' # tampoc dona, però en la 15 l'hem redefinifit i ja dona bé. Apareix algun error a l'hora de resoldre-la aixina
f14='diff(f(x),x) - (2*x -5*f(x)+3)/(2*x +4*f(x)-6)'
f15='diff(f(x),x)-6*x^3 - 3*f(x)/x'


f=f15
f = sp.S(f, locals=env.local_dict)
f           # el input que vamos a meterle al modelo para que calcule la solucion

-6*x**3 + Derivative(f(x), x) - 3*f(x)/x

In [ ]:
display('Ecuaciones diferenciales de primer orden que he testeado:')
display(sp.S(f1, locals=env.local_dict))
print("")
display(sp.S(f2, locals=env.local_dict))
print("")
display(sp.S(f3, locals=env.local_dict))
print("")
display(sp.S(f4, locals=env.local_dict))
print("")
display(sp.S(f5, locals=env.local_dict))
print("")
display(sp.S(f6, locals=env.local_dict))
print("")
display(sp.S(f7, locals=env.local_dict))
print("")
display(sp.S(f8, locals=env.local_dict))
print("")

'Ecuaciones diferenciales de primer orden que he testeado:'

f(x)*sin(x) + cos(x)*Derivative(f(x), x) - 1

-2*(cos(x) - 1)*exp(2*sin(x)) - 2*f(x) + Derivative(f(x), x)

x**2*Derivative(f(x), x) + x*f(x) + sqrt(f(x))

2*f(x)*Derivative(f(x), x) - 2*f(x)/cos(x)**2 + 2*f(x)**2/x

x**2*(f(x)**2 + Derivative(f(x), x)) + x*f(x) - 1

-2*f(x)/cos(x)**2 + Derivative(f(x), x) + 2*f(x)/x

2*x + (x - 2*f(x))*Derivative(f(x), x) + f(x)

-x**2*f(x) + x*Derivative(f(x), x) - exp(x**2/2)

In [ ]:
#F_prefix = env.sympy_to_prefix(F)     # cambiamos la notación a sequencias para el modelo seq2seq
f_prefix = env.sympy_to_prefix(f) 
s = f_prefix
#print(f"F prefix: {F_prefix}")
print(f"f prefix: {f_prefix}")

f prefix: ['add', 'mul', 'INT-', '6', 'pow', 'x', 'INT+', '3', 'add', 'mul', 'INT-', '3', 'mul', 'pow', 'x', 'INT-', '1', 'f', 'x', 'derivative', 'f', 'x', 'x']


In [ ]:
#ODE_prefix = ['add','derivative', 'f', 'x', 'x','mul','x','f','x']
#x1_prefix = env.clean_prefix(ODE_prefix)
#x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
x1_prefix =env.clean_prefix(f_prefix)  # simplificamos al máximo la EDO
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +      
    [env.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1) 

In [ ]:

#para comprobar lo que he escrito en el prefix
#ODE_infix = env.prefix_to_infix(ODE_prefix)
#ODE_infix = env.prefix_to_infix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
#ODE_sympy = env.infix_to_sympy(ODE_infix)
#print(ODE_infix)
#display(ODE_sympy)

In [ ]:
beam_size = 5  #numero de hypotesis que queremos
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

In [ ]:
print(f"Input function f: ")
display(f)
#print(f"Reference function F: {F}")
print("")

hyps=[]
scores=[]
results=[]
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    scores.append(score)
    
    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix
    

    try:
        hyp = env.prefix_to_infix(tok)
               # convert to infix
        hyp = env.infix_to_sympy(hyp) 
        #print(checkodesol(f, hyp))
        simp= checkodesol(f, hyp)[1]
        

        
        res = "CORRECT" if simp == 0 else "INCORRECT"  
        for i in tok :
         if res == 'INCORRECT' and (i =='sqrt' or i =='ln') :
            res = res + '\n WARNNING : el modelo tiene problemas con la simplifación de algunas funciones (raízes, logaritmos, etc..) puede que el resultado sea correcto y que la solución no esté simplificada'

       # si la sustitució de hyp en f dona zero, diguem que el resultat és correcte

        
    except:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)

        res = "INVALID PREFIX EXPRESSION" 
    results.append(res)
    hyps.append(hyp)
    
    # print result
    display(hyp)
    print("Score: %.5f" % score )
    print("%s RESULT" % res)
    print(' Evaluamos la hipotesis generada en la función input y el resultado es : ')
    display(simp)
    print('\n\n Next hypothesis:')

Input function f: 


-6*x**3 + Derivative(f(x), x) - 3*f(x)/x

2*x**3*log(x**2*exp(a8 + 3*x))

Score: -0.16884
INCORRECT
 WARNNING : el modelo tiene problemas con la simplifación de algunas funciones (raízes, logaritmos, etc..) puede que el resultado sea correcto y que la solución no esté simplificada RESULT
 Evaluamos la hipotesis generada en la función input y el resultado es : 


4*x**3

...



 Next hypothesis:


2*x**2*(3*x*(a8 + x) + 2*x)

Score: -0.27658
CORRECT RESULT
 Evaluamos la hipotesis generada en la función input y el resultado es : 


0



 Next hypothesis:


In [ ]:
import pandas as pd
df=pd.DataFrame([results,scores,hyps]).T
df.columns=['test_result','score','function']
df

,test_result,score,function
0,INCORRECT\n WARNNING : el modelo tiene problem...,-0.168843,2*x**3*log(x**2*exp(a8 + 3*x))
1,INCORRECT\n WARNNING : el modelo tiene problem...,-0.23058,2*x**3*log(x**2*exp(a8 + x))
2,INCORRECT\n WARNNING : el modelo tiene problem...,-0.262238,2*x**3*log(x**2*exp(a8 - x))
3,CORRECT,-0.270346,2*x**2*(a8*x + 3*x**2)
4,CORRECT,-0.27658,2*x**2*(3*x*(a8 + x) + 2*x)


In [ ]:
xx= df.iloc[0,2]
xx


2*x**3*log(x**2*exp(a8 + 3*x))

In [ ]:
######  #####


# a partir d'ací no es codi, son proves per entendre coses 



prova = env.prefix_to_infix(['add', 'x', 'add', 'mul', 'a8', 'x', 'mul', 'INT-', '1', 'mul', 'x', 'ln', 'mul', 'INT+', '2', 'x'])


ODE_prova = env.infix_to_sympy(prova)
print(prova)
display(ODE_prova)

mmm= env.prefix_to_infix(['mul', 'x', 'add', 'a8', 'mul', 'INT-', '1', 'ln', 'mul', 'div', 'INT+', '1', 'INT+', '2', 'x'])

rr = env.infix_to_sympy(mmm)
print(mmm)
display(rr)



((x)+(((a8)*(x))+((-1)*((x)*(ln((2)*(x)))))))


a8*x - x*log(2*x) + x

((x)*((a8)+((-1)*(ln(((1)/(2))*(x))))))


x*(a8 - log(x/2))

In [ ]:
x1_prefix

# hemos simplificado la ecuacion  para introducirla a el modelo. la ecuacion se ha simplificado a '

In [ ]:
### NOTASSS##
# he arribat a conseguir que el model prediga les solucions correctes
# ara falta crear un algoritme per a verificar que les solucions son correctes  ---> ja fetv
# la idea es sustituir la solcuió en l'EDO  vore q , efectivament, dona zero   --> ja fet

# Per a expressions senzilles dona bons resultats, però quan es compliquen les expressions el procés de comprovació falla i necessita molt de temps per a compilar

In [ ]:
from sympy.solvers.ode.ode import ode_sol_simplicity
#f1.subs(f(x),hyp)
#checkodesol(f, xx)
#ode_sol_simplicity(xx,f_prefix)


In [ ]:
## Per a fer 

#warning de que no saca soluciones para raizes
# documentar todos los errores